<a href="https://colab.research.google.com/github/pedr0-almeida/AI2023/blob/main/Tarefa_01_placa_carros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage
import tensorflow as tf
from sklearn.model_selection import train_test_split
from imageio import imread
from google.colab import files
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
import pytesseract
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
!gdown 17DeutH_Rzmw7J6Qdr9XdMFTRSH8v7VvY

Downloading...
From: https://drive.google.com/uc?id=17DeutH_Rzmw7J6Qdr9XdMFTRSH8v7VvY
To: /content/licence_plate.zip
100% 2.96M/2.96M [00:00<00:00, 175MB/s]


In [ ]:
FILE = 'licence_plate.zip'
archive = zipfile.ZipFile(FILE, 'r')
archive.extractall()

In [ ]:
carro = []
placa = []


In [ ]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [ ]:
for word in img_name:
  img = cv2.imread(word)
  cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  test, bin = cv2.threshold(cinza, 100, 255, cv2.THRESH_BINARY)
  desfoque = cv2.GaussianBlur(bin, (5, 5), 0)
  contornos, hierarquia = cv2.findContours(desfoque, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
  cv2.drawContours(img, contornos, -1, (90, 255, 0), 1)
 
  for c in contornos:
    perimetro = cv2.arcLength(c, True)
    if perimetro > 120:
        aprox = cv2.approxPolyDP(c, 0.03 * perimetro, False)
        if len(aprox) == 4:
            (x, y, alt, lar) = cv2.boundingRect(c)
            cv2.rectangle(img, (x, y), (x + alt, y + lar), (0, 255, 0), 2)
            roi = img[y:y + lar, x:x + alt]
 
  resize_img_roi = cv2.resize(roi, None, fx=4, fy=4, interpolation=cv2.INTER_CUBIC)
  #cv2_imshow(resize_img_roi)
  img_cinza = cv2.cvtColor(resize_img_roi, cv2.COLOR_BGR2GRAY)
  _, img_binary = cv2.threshold(img_cinza, 70, 255, cv2.THRESH_BINARY)
  img_desfoque = cv2.GaussianBlur(img_binary, (5, 5), 0)
  img_ocr = img_desfoque
  #cv2_imshow(img_ocr)
  config = r'-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 --psm 6'
  texto = pytesseract.image_to_string(img_ocr, lang='eng', config=config)
  placa.append(texto)
  carro.append(word)
  #print(texto)
  #cv2_imshow(img)

In [ ]:
df = pd.DataFrame({'img':carro, 'placa':placa})
df.to_excel('Placas.xlsx')
#files.download('Placas.xlsx')
df

,img,placa
0,placa_carro3.jpg,NN N\n
1,placa_carro2.jpg,VT7\nS\nSS SS\n4 4 LA\n1 FAY 3 WAS A 7\nW 7 V ...
2,00011.jpg,Y A\n
3,00024.jpg,Y A\n
4,00029.jpg,SO\n
5,car_0001.jpg,RE\n
6,car_0002.jpg,RE\n
7,car_0004.jpg,RE\n
8,car_0005.jpg,RE\n
9,car_0006.jpg,RE\n
